<div style="width: 30%; float: right; margin: 10px; margin-right: 5%;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/FHNW_Logo.svg/2560px-FHNW_Logo.svg.png" width="500" style="float: left; filter: invert(50%);"/>
</div>

<h1 style="text-align: left; margin-top: 10px; float: left; width: 60%;">
    npr Mini-Challenge 2 <br> 
</h1>

<p style="clear: both; text-align: left;">
    Bearbeitet durch Si Ben Tran, Yannic Lais, Rami Tarabishi im HS 2023.<br>Bachelor of Science FHNW in Data Science.
</p>


# Aufgabe aus dem Space
Mini-Challenge 2-C (LE6):
Chatbot

You build a simple chatbot for hotel recommendation (here you can combine with npr and webscraping challenge) or extend the chatbot from the library (auxilio, please take contact with me), with rasa or dialogflow. You can also propose a use case. Important is the way intent are recognized and NER or variables are extracted. Delivery is a report on the use case, interactions and how intents are recognized and variables extracted. Also an error analysis on concrete conversations (confidence of models, explanation of predictions, etc.) should be undertaken. The submission can be done in a group of 2 or 3.


# Aufgabenstellung

Wir haben hier einen simplen ChatBot erstellt, welches die Anfrage des Benutzers entgegennimmt und eine Antwort zurückgibt. Die Anfrage des Benutzers wird durch unsere Preprocessing Pipeline bearbeitet und anschliessend durch ein Machine Learning Model (Kleines Neuronales Netz) klassifiziert - Intent Klassifikation. Anschliessend wird eine zufällige Antwort aus einer Liste von Antworten zurückgegeben. Weiter haben wir versucht die NER (Named Entity Recognition) zu implementieren, welche aber nicht so gut funktioniert hat.

# 1 Libraries

In [1]:
%load_ext autoreload
%autoreload 2

import json
import numpy as np
from tqdm import tqdm 

import nltk
# nltk.download('punkt')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import spacy
spacy_nlp = spacy.load("en_core_web_sm")

from src.preprocessing import tokenize, stem, bag_of_words
from src.news_scraper import scrape_srf_links, get_data_for_topics

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preprocessing with bert Transformer
from transformers import BertTokenizer, BertModel
from transformers import BertModel
from torch.optim import Adam
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader
from tqdm import tqdm



# 2 Create Data 

In [2]:
# Scrape srf webpage
url = "https://www.srf.ch"
topic_list = scrape_srf_links(url)
# create data 
get_data_for_topics(topic_list)

Combined JSON file 'data/intents_news.json' created successfully for all topics.


In [3]:
with open('data/intents_news.json', 'r') as f:
    intents_news = json.load(f)

with open('data/intents-scratch.json', 'r') as f:
    intents = json.load(f)

# combine intents_news with intens
intents["intents"].extend(intents_news["intents"])

# export intents formmated as json
with open('data/intents.json', 'w') as outfile:
    json.dump(intents, outfile, indent=4)

In [4]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents["intents"]:
    tag = intent["tag"]
    # add to tag list
    tags.append(tag)
    for pattern in intent["patterns"]:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

print(all_words)
print(tags)
print(xy)

['Hello', ',', 'my', 'name', 'is', 'Hi', ',', 'I', "'m", 'Hey', ',', 'I', 'go', 'by', 'I', 'am', 'Greetings', ',', 'call', 'me', 'You', 'can', 'call', 'me', 'They', 'call', 'me', 'My', 'name', "'s", 'I', "'d", 'like', 'to', 'introduce', 'myself', ',', 'I', "'m", 'Allow', 'me', 'to', 'introduce', 'myself', ',', 'I', 'am', 'I', 'go', 'by', 'the', 'name', 'People', 'know', 'me', 'as', 'Hi', 'Hey', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Whats', 'up', 'How', 'are', 'you', 'doing', 'Howdy', 'Hi', 'there', 'Hola', 'Good', 'morning', 'Good', 'afternoon', 'Good', 'evening', 'It', 'is', 'nice', 'to', 'meet', 'you', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Have', 'a', 'nice', 'day', 'See', 'you', '!', 'Later', '!', 'Goodbye', '!', 'I', "'m", 'off', '!', 'That', "'s", 'all', 'for', 'now', '.', 'See', 'you', 'soon', '!', 'I', "'ve", 'got', 'to', 'get', 'going', '.', 'Bye', '!', 'It', 'was', 'n

# 3 Bert Tokenizer

In [5]:
# Assume 'tags' and 'xy' are predefined
# Bert Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Init model
model = BertModel.from_pretrained('bert-base-uncased', len(tags))

# encode xy pairs
X_train = []
y_train = []

for (pattern_sentence, tag) in xy:
    # encode sentence
    encoded_sentence = tokenizer(
        ' '.join(pattern_sentence),
        max_length=16,
        padding='max_length',
        return_tensors='pt',
        truncation=True
    )

    # Append only the required tensors
    X_train.append((encoded_sentence['input_ids'].squeeze(0), encoded_sentence['attention_mask'].squeeze(0)))
    y_train.append(tags.index(tag))

# create dataloaders
train_data = list(zip(X_train, y_train))
train_loader = DataLoader(train_data, batch_size=16, num_workers=0)

In [6]:
# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cpu



In [7]:
class BertClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BertClassifier, self).__init__()
        self.bert = bert_model
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        return self.classifier(pooled_output)

# Assuming 'tags' is a list of unique classes
model = BertClassifier(BertModel.from_pretrained('bert-base-uncased'), len(tags))

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-5)
num_epochs = 10

In [11]:
# Training loop
model.train()
for epoch in range(num_epochs):  # num_epochs should be defined by you
    # Initialize tqdm progress bar
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # Unpack the batch
        (input_ids, attention_mask), labels = batch

        # Move tensors to the appropriate device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update progress bar
        loop.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
        loop.set_postfix(loss=loss.item())

    loop.close()


Epoch [10/10]: 100%|██████████| 29/29 [01:02<00:00,  2.16s/it, loss=3.31]


# 4 Save Model

In [12]:
# save model
MODEL_PATH = 'model/bert_model.pth'
torch.save(model.state_dict(), MODEL_PATH)
print(f'training complete. file saved to {MODEL_PATH}')

training complete. file saved to model/bert_model.pth


# 5 Load Model

In [13]:
# Load Model
model = BertClassifier(BertModel.from_pretrained('bert-base-uncased'), len(tags))
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()


BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

# 6 Predict Intent and get Response

In [14]:
print("Heutige News Topic:", scrape_srf_links(url))

Heutige News Topic: {'school', 'podcasts', 'www.srf.ch', 'gesellschaft', 'www.swissinfo.ch', 'www.srgssr.ch', 'www.radioswissjazz.ch', 'www.rsi.ch', 'literatur', 'gesellschaft-religion', 'jobs.srf.ch', 'mehr-sport', 'film-serien', 'gesichterundgeschichten', 'wetter', 'schweiz', 'website-und-apps', 'kunst', 'www.rtr.ch', 'startseite', 'international', 'ski-alpin', 'decodar-nossa-cultura', 'www.radioswisspop.ch', 'wirtschaft', 'dialog', 'fussball', 'radio', 'mensch', 'gesundheit', 'www.radioswissclassic.ch', 'eishockey', 'tv', 'woerterbuch', 'www.rts.ch', 'musik', 'allgemein', 'kassensturz-espresso', 'video', 'themen'}


In [15]:
def predict_intent(sentence):
    """
    Predicts the intent of a sentence using the trained model.
    """
    # Encode the sentence
    encoded_sentence = tokenizer(
        sentence,
        max_length=16,
        padding='max_length',
        return_tensors='pt',
        truncation=True
    )
    input_ids = encoded_sentence['input_ids'].squeeze(0)
    attention_mask = encoded_sentence['attention_mask'].squeeze(0)

    # Make prediction
    with torch.no_grad():
        outputs = model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0))
    prediction = torch.argmax(outputs, dim=1)
    # give me a random answer based in the prediction
    reponse = np.random.choice(intents["intents"][prediction.item()]["responses"])

    return reponse

# 7 Chatbot

In [16]:
bot_name = "News Assisstent Bot"
while True:
    sentence = input("You: ")
    print("User Input:", sentence)
    if sentence == "quit":
        break

    # get response
    response = predict_intent(sentence)
    print(f"{bot_name}: {response}")

User Input: school
News Assisstent Bot: Farewell!
User Input: hi
News Assisstent Bot: Bye! See you later.
User Input: 
News Assisstent Bot: Bye! Have a nice day.
User Input: bye
News Assisstent Bot: See you later, thanks for visiting
User Input: Hey how are you?
News Assisstent Bot: Farewell!
User Input: hi
News Assisstent Bot: Farewell!
User Input: quit
